In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

In [25]:
df_s = pd.read_csv('data/spatial-cdf-ma-test-shape.csv')
# for district in districts: 
#     for attr in ['BF', 'FC', 'CHH', 'MSL', 'MSW']:
#         for lvl in ['RUD', 'INT', 'ADV']:
#             x = df_s.loc[df_s['District'] == district, attr+'_'+lvl]
#             x = (x - min(x))/(max(x) - min(x))
#             df_s.loc[df_s['District'] == district, attr+'_'+lvl] = x
#     print(district, ' done')
df_master = pd.read_csv('data/master-all-districts.csv')
df_master.columns = [ 'FC_Change' if x == 'FC_Chnage' else x for x in df_master.columns]
print(np.asarray(df_master.columns))
df_master = df_master[df_master['District'].isin(pd.unique(df_s['District']))]
districts = pd.unique(df_s['District'])
print(len(pd.unique(df_s['District'])))
df_temp = df_s[np.logical_not(df_s['District'].isin(pd.unique(df_master['District'])))]
districts_not_to_count = pd.unique(df_temp['District'])

['STATE' 'Name' 'NAME' 'District' 'No_HH' 'TOT_P' 'ELG_POP' 'LIT' 'F_EMP'
 'F_EMP_Female' 'EMP_AL' 'EMP_NAL' 'EMP_UN' 'FC_RUD' 'FC_INT' 'FC_ADV'
 'MSL_RUD' 'MSL_INT' 'MSL_ADV' 'CHH_RUD' 'CHH_INT' 'CHH_ADV' 'MSW_RUD'
 'MSW_INT' 'MSW_ADV' 'BF_RUD' 'BF_INT' 'BF_ADV' 'radio/transistor' 'TV'
 'Telephone' 'Bicycle' '2 wheeler' '4 wheeler' 'No asset' 'State'
 'District_2011' 'No_HH_2011' 'TOT_P_2011' 'EMP_AL_2011' 'EMP_NAL_2011'
 'EMP_UN_2011' 'ELG_POP_2011' 'LIT_2011_val' 'F_EMP_2011' 'BF_RUD_2011'
 'BF_INT_2011' 'BF_ADV_2011' 'CHH_RUD_2011' 'CHH_INT_2011' 'CHH_ADV_2011'
 'FC_RUD_2011' 'FC_INT_2011' 'FC_ADV_2011' 'MSL_RUD_2011' 'MSL_INT_2011'
 'MSL_ADV_2011' 'MSW_RUD_2011' 'MSW_INT_2011' 'MSW_ADV_2011' 'Radio_2011'
 'TV_2011' 'Telephone_2011' 'Cycle_2011' '2 Wheeler_2011' '4 Wheeler_2011'
 'No Asset_2011' 'EMP_2011' 'EMP_2001' 'MSW_2011' 'MSW_2001' 'BF_2011'
 'BF_2001' 'MSL_2011' 'MSL_2001' 'FC_2011' 'FC_2001' 'CHH_2011' 'CHH_2001'
 'Population Change' 'MSL_Change' 'MSW_Change' 'FC_Change' '

In [68]:
pd.unique(df_master['MSL_Change'])

array([0, 1])

In [7]:
def fill_table(df, to_predict='2011_label'):
    for attr in ['BF', 'CHH', 'FC', 'MSL', 'MSW']:
        df_master.sort_values('District', inplace=True)
        if to_predict == '2011_label':
            y = df_master[attr+'_2011']
        if to_predict == 'change':
            y = df_master[attr+'_Change']
        for lvl in ['RUD', 'INT', 'ADV', 'All']:
            print(lvl)
            X = []
            districts = pd.unique(df_s['District'])
            districts = np.sort(districts)
            for district in districts:
                if district not in districts_not_to_count:
                    if lvl != 'All':
                        X.append(np.asarray(df_s.loc[df_s['District'] == district, attr+'_'+lvl]))
                    else:
                        single_datapoint = np.concatenate((np.asarray(df_s.loc[df_s['District'] == district, attr+'_RUD']), 
                                                           np.asarray(df_s.loc[df_s['District'] == district, attr+'_INT']), 
                                                           np.asarray(df_s.loc[df_s['District'] == district, attr+'_ADV'])), axis=None)
                        X.append(single_datapoint)

            X = np.asarray(X)
            clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
            clf1 = DummyClassifier(strategy='stratified')
            print(cross_val_score(clf, X, y, cv=5))
            df.loc[lvl, (attr, 'LogReg', 'Accuracy')] = round(np.mean(cross_val_score(clf, X, y, cv=5)), 2)
            df.loc[lvl, (attr, 'LogReg', 'F1')] = round(np.mean(cross_val_score(clf, X, y, cv=5, scoring='f1_macro')), 2)
            df.loc[lvl, (attr, 'Dummy', 'Accuracy')] = round(np.mean(cross_val_score(clf1, X, y, cv=5)), 2)
            df.loc[lvl, (attr, 'Dummy', 'F1')] = round(np.mean(cross_val_score(clf1, X, y, cv=5, scoring='f1_macro')), 2)
            print(attr, ', ', lvl, ' done')
    #         print(round(np.mean(cross_val_score(clf, X, y, cv=5)), 2))
    #         print('Dummy - ')
    #         print(cross_val_score(clf1, X, y, cv=5))
    #         print(round(np.mean(cross_val_score(clf1, X, y, cv=5)), 2))

In [28]:
iterables = [['BF', 'CHH', 'FC', 'MSL', 'MSW'], ['LogReg', 'Dummy'], ['Accuracy', 'F1']]
pd.MultiIndex.from_product(iterables, names=['indicator', 'model', 'metric'])
df_2011 = pd.DataFrame(index=['RUD', 'INT', 'ADV', 'All'], columns=pd.MultiIndex.from_product(iterables, names=['indicator', 'model', 'metric']))
fill_table(df_2011, '2011_label') 
df_change = pd.DataFrame(index=['RUD', 'INT', 'ADV', 'All'], columns=pd.MultiIndex.from_product(iterables, names=['indicator', 'model', 'metric']))
fill_table(df_change, 'change')

RUD
[0.65714286 0.64705882 0.63636364 0.78125    0.90625   ]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:113

BF ,  RUD  done
INT
[0.68571429 0.58823529 0.66666667 0.5        0.71875   ]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:113

BF ,  INT  done
ADV
[0.71428571 0.79411765 0.54545455 0.6875     0.8125    ]


...

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:113

MSW ,  INT  done
ADV
[0.91176471 0.90909091 0.90909091 0.90909091 0.90909091]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:113

MSW ,  ADV  done
All
[0.91176471 0.87878788 0.90909091 0.90909091 0.66666667]
MSW ,  All  done


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T

In [29]:
print(df_2011)
# df.to_csv('predicting-2001-label-from-spatial-curves.csv')

indicator       BF                           CHH                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD           0.73  0.39     0.61  0.27     0.48  0.44      0.3  0.32   
INT           0.63  0.35     0.54  0.34     0.47  0.44     0.31  0.34   
ADV           0.71  0.43     0.55  0.35     0.48  0.43     0.34  0.29   
All           0.71  0.42     0.44  0.28     0.47  0.44     0.28  0.32   

indicator       FC                           MSL                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD           0.67  0.48     0.51  0.39     0.75  0.57     0.43   0.3   
INT            0.6  0.42     0.46  0.34     0.76  0.62     0.39  0.32   
ADV           0.51  0.33     0.51  0.35     0.67   0.5     0.38  0.28   
All           0.66  0.55     0.43  0.28      0.8  

In [89]:
fill_table(df_change, 'change')

RUD
[0.79411765 0.84848485 0.78787879 0.81818182 0.36363636]


/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: 

BF ,  RUD  done
INT
[0.82352941 0.87878788 0.78787879 0.78787879 0.72727273]


/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: 

BF ,  INT  done
ADV
[0.82352941 0.93939394 0.63636364 0.81818182 0.48484848]


...

/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: 

MSW ,  INT  done
ADV
[0.91176471 0.90909091 0.66666667 0.87878788 0.87878788]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy

MSW ,  ADV  done
All
[0.88235294 0.72727273 0.90909091 0.72727273 0.72727273]
MSW ,  All  done
indicator       BF                           CHH                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD           0.72  0.51     0.72  0.48     0.72  0.71     0.52  0.43   
INT            0.8  0.61     0.74  0.52     0.56  0.51     0.55  0.52   
ADV           0.74  0.63     0.66  0.49     0.68  0.65     0.52  0.45   
All           0.76  0.62     0.68  0.45     0.71  0.69     0.53  0.52   

indicator       FC                           MSL                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD           0.81  0.45     0.86  0.49     0.68  0.55     0.64  0.45   
INT           0.88  0.55     0.85  0.55     0.72  0.66     0.59  0.46   
ADV           0.86  0.57    

/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  k in range(self.n_outputs_)).T
/usr/local/lib/python3.7/site-packages/sklearn/dummy.py:227: FutureWarning: 

In [32]:
print(df_change)

indicator       BF                           CHH                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD           0.73  0.42     0.75  0.49     0.63  0.54     0.49  0.54   
INT           0.77  0.51     0.69  0.54     0.59  0.54     0.49  0.48   
ADV            0.8  0.51      0.7  0.54     0.64  0.56     0.49  0.51   
All           0.79   0.6     0.69  0.47     0.64   0.6     0.51  0.51   

indicator       FC                           MSL                       \
model       LogReg          Dummy         LogReg          Dummy         
metric    Accuracy    F1 Accuracy    F1 Accuracy    F1 Accuracy    F1   
RUD            0.9  0.47     0.79  0.53     0.61   0.4     0.61  0.46   
INT            0.9  0.47     0.81  0.53     0.62  0.45     0.57  0.53   
ADV            0.9  0.47     0.81  0.53     0.67  0.47      0.6  0.49   
All           0.88  0.47     0.83  0.51     0.61  

In [33]:
df_2011.to_csv('predicting-2011-label-from-spatial-shape-curves.csv')
df_change.to_csv('predicting-change-label-from-spatial-shape-curves.csv')

In [27]:
# x = df_s.loc[df_s['District'] == 203, 'BF_RUD']
for district in districts: 
    for attr in ['BF', 'FC', 'CHH', 'MSL', 'MSW']:
        for lvl in ['RUD', 'INT', 'ADV']:
            x = df_s.loc[df_s['District'] == district, attr+'_'+lvl]
            if (max(x) - min(x)) > 0:
                x = (x - min(x))/(max(x) - min(x))
            else:
                x = max(x)
            df_s.loc[df_s['District'] == district, attr+'_'+lvl] = x
    print(district, ' done')

203  done
204  done
205  done
206  done
207  done
208  done
209  done
210  done
211  done
212  done
213  done
214  done
215  done
216  done
217  done
218  done
219  done
220  done
221  done
222  done
223  done
224  done
225  done
226  done
227  done
228  done
229  done
230  done
231  done
232  done
233  done
234  done
235  done
236  done
237  done
238  done
239  done
240  done
370  done
371  done
372  done
373  done
374  done
375  done
376  done
377  done
378  done
379  done
380  done
381  done
382  done
383  done
384  done
385  done
386  done
387  done
388  done
389  done
390  done
391  done
392  done
393  done
394  done
395  done
396  done
397  done
398  done
399  done
468  done
469  done
470  done
471  done
472  done
473  done
474  done
475  done
476  done
477  done
478  done
479  done
480  done
481  done
482  done
483  done
484  done
485  done
486  done
487  done
488  done
489  done
490  done
491  done
492  done
493  done
497  done
498  done
499  done
500  done
501  done
502  done


In [26]:
df_s

,Unnamed: 0,From,To,Mid,District,Floor,Top,State,BF_RUD,BF_INT,...,FC_ADV,MSW_RUD,MSW_INT,MSW_ADV,MSL_RUD,MSL_INT,MSL_ADV,EMP_UN,EMP_AL,EMP_NAL
0,0,0,0.016949,0.008475,203,1.242869,2.969447,10,0.875000,0.000000,...,0.125000,0.000000,1.000000,0.000000,0.875000,0.083333,0.041667,0.875000,0.041667,0.083333
1,1,0,0.033898,0.016949,203,1.242869,3.597471,10,0.893617,0.000000,...,0.106383,0.000000,1.000000,0.000000,0.829787,0.127660,0.042553,0.808511,0.085106,0.106383
2,2,0,0.050847,0.025424,203,1.242869,4.478905,10,0.873239,0.014085,...,0.084507,0.000000,1.000000,0.000000,0.830986,0.126761,0.042254,0.830986,0.070423,0.098592
3,3,0,0.067797,0.033898,203,1.242869,4.992198,10,0.893617,0.010638,...,0.063830,0.010638,0.989362,0.000000,0.840426,0.117021,0.042553,0.829787,0.095745,0.074468
4,4,0,0.084746,0.042373,203,1.242869,5.700996,10,0.905983,0.008547,...,0.051282,0.008547,0.991453,0.000000,0.863248,0.102564,0.034188,0.820513,0.119658,0.059829
5,5,0,0.101695,0.050847,203,1.242869,6.286711,10,0.921986,0.007092,...,0.042553,0.007092,0.992908,0.000000,0.865248,0.106383,0.028369,0.801418,0.148936,0.049645
6,6,0,0.118644,0.059322,203,1.242869,6.755488,10,0.932927,0.006098,...,0.036585,0.006098,0.993902,0.000000,0.878049,0.097561,0.024390,0.786585,0.164634,0.048780
7,7,0,0.135593,0.067797,203,1.242869,7.243949,10,0.941176,0.005348,...,0.032086,0.005348,0.994652,0.000000,0.893048,0.085561,0.021390,0.786096,0.171123,0.042781
8,8,0,0.152542,0.076271,203,1.242869,7.626798,10,0.947867,0.004739,...,0.028436,0.004739,0.995261,0.000000,0.905213,0.075829,0.018957,0.786730,0.175355,0.037915
9,9,0,0.169492,0.084746,203,1.242869,8.155554,10,0.952991,0.004274,...,0.025641,0.004274,0.995726,0.000000,0.914530,0.068376,0.017094,0.786325,0.179487,0.034188
